In [2]:
from dataset import AarizDataset

path = "Aariz Dataset"
dataset = AarizDataset( path ,"TRAIN")

In [3]:
image, landmarks, cvm = dataset[0]

Dimension of array is (2225, 1968, 3)


In [4]:
print(image.size())
print(landmarks.size())




torch.Size([2225, 1968, 3])
torch.Size([1, 29, 2])


In [6]:
from PIL import Image
import os


folder = "Aariz Dataset/train/Cephalograms"

max_width = max_height = 0


for f in os.listdir(folder):
    if f.lower().endswith(".png"):
        with Image.open(os.path.join(folder, f)) as img:
            w, h = img.size
            max_width = max(max_width, w)
            max_height = max(max_height, h)

print("Max width:", max_width)
print("Max height:", max_height)

Max width: 2114
Max height: 2234
